# 22.目标导向的强化学习（goal-oriented reinforcement learning，GoRL）
> 之前学习的如 **PPO、SAC** 等经典的深度强化学习算法，智能体的行为通常是根据固定的奖励函数来学习的，只关注单一目标。算法训练完后，在使用它们时只能完成一个特定的任务。
> 而在许多实际任务场景中，目标是复杂的，涉及到 **多个子目标或必须在长期内达成的目标**。之前传统的强化学习算法往往不容易训练出有效的策略。
> 例如，在悬崖漫步环境中，若目标位置变成了右上角，便只能重新训练一个策略。同一个策略无法完成一系列不同的目标。

> **目标导向的强化学习（GoRL）** 的概念并非源自单一的原始论文，而是逐渐在多个领域中演化和扩展的一个 **思想**：
> - 让智能体在前往 **最终目标** 的训练过程中将所经历的 **失败经验** 也作为一种 **目标** 进行学习，使智能体理解 **错误目标** 是如何达成的，最终学习一种 **策略**：在应对 **不同的目标（goal）** 依然奏效。以此来解决较为复杂的 **复合任务**。

> 本节介绍该思想下的一种经典算法 [事后经验回放（hindsight experience replay，HER）](https://arxiv.org/abs/1707.01495)：主要是为了解决 **稀疏奖励**

## 22.1 GoRL 中的马尔可夫决策过程（MDP）
> 在 **目标导向的强化学习** 中，**MDP** 的定义使用一个扩充过的元组：
> $$\langle\mathcal{S},\mathcal{A},P,r_{g},\mathcal{G},\phi\rangle$$

- **$\mathcal{S}$**：状态空间（state space），环境中所有可能的状态
- **$\mathcal{A}$**：动作空间（action space），智能体在每个状态下可采取的动作集合
- **$P$**：状态转移概率分布 $P(s'|s,a)$，描述动作如何影响状态变化
- **$r_g$**：**目标条件奖励函数**，即奖励不仅依赖于状态和动作，还依赖于当前目标 $g \in \mathcal{G}$
- **$\mathcal{G}$**：目标空间（goal space），任务中所有可能的目标集合
- **$\phi$**：目标映射函数，将状态 $s \in \mathcal{S}$ 映射到目标相关的表示 $\phi(s) \in \mathcal{G}$，用于判断某个状态是否达成目标
> 可见，这是一个 **带目标条件的 MDP**（Goal-conditioned MDP，GC-MDP）。相比于传统 RL 只关心“最大化奖励”，在 GoRL 中，**奖励函数和策略都依赖于目标**，比如目标是“到达某个位置”，那么奖励设计和学习策略都随目标改变。因此 **状态价值函数$V(s,g)$** 也是基于目标的，**动作状态价值函数$Q(s,a,g)$** 同理。

> 举个例子：**机器人导航**
- $\mathcal{S}$：房间里机器人的位置（状态空间）
- $\mathcal{A}$：机器人能走的动作（上下左右）
- $P$：走一步后到达新位置的概率
- $r_g$：如果机器人到达目标位置 $g$，奖励 = 1，否则 = 0
- $\mathcal{G}$：房间里所有可能的目标位置 $g_i$
- $\phi(s)$：提取当前位置，判断是否等于目标

> 如此，为了机器人能够能根据不同的目标 $g$ 导航到对应位置，**GoRL** 的目标为优化一个 **通用的策略 $\pi(a|s,g)$**，使以下目标函数最大化：
$$\mathbb{E}_{(s_0,g)\sim\nu_0}\left[V^\pi\left(s_0,g\right)\right]$$
- $\nu_0$ 为环境中初始状态 $s_0$ 与目标 $g$ 的联合分布

## 22.2 HER 算法